# 第14章: 継承とポリモーフィズム - 解答集

この解答集では、課題の解答と詳細な解説を提供します。

**重要**: 自分で解いてから解答を見るようにしてください！

**解答は容量制限のため、主要な課題の要点を解説します。**

---
## 課題1: 支払いシステム（基礎レベル）- 解答

### 主要クラスの実装

In [ ]:
<?php

/**
 * PaymentMethodインターフェース
 */
interface PaymentMethod {
    public function pay(int $amount): bool;
    public function getName(): string;
    public function isAvailable(): bool;
}

/**
 * CreditCardクラス
 */
class CreditCard implements PaymentMethod {
    private string $cardNumber;
    private string $expiryDate;
    private string $cvv;
    private int $creditLimit = 100000;
    
    public function __construct(string $cardNumber, string $expiryDate, string $cvv) {
        $this->cardNumber = $cardNumber;
        $this->expiryDate = $expiryDate;
        $this->cvv = $cvv;
    }
    
    public function pay(int $amount): bool {
        if (!$this->isValidCard()) {
            return false;
        }
        
        if ($amount > $this->creditLimit) {
            echo "与信枠超過\n";
            return false;
        }
        
        echo "クレジットカードで" . number_format($amount) . "円支払いました\n";
        return true;
    }
    
    public function getName(): string {
        $masked = str_repeat('*', 12) . substr($this->cardNumber, -4);
        return "クレジットカード ({$masked})";
    }
    
    public function isAvailable(): bool {
        return $this->isValidCard();
    }
    
    private function isValidCard(): bool {
        return strlen($this->cardNumber) === 16 && ctype_digit($this->cardNumber);
    }
}

/**
 * BankTransferクラス
 */
class BankTransfer implements PaymentMethod {
    private string $bankName;
    private string $accountNumber;
    private int $balance = 500000;
    
    public function __construct(string $bankName, string $accountNumber) {
        $this->bankName = $bankName;
        $this->accountNumber = $accountNumber;
    }
    
    public function pay(int $amount): bool {
        if ($amount > $this->balance) {
            echo "口座残高不足\n";
            return false;
        }
        
        echo "銀行振込({$this->bankName})で" . number_format($amount) . "円支払いました\n";
        $this->balance -= $amount;
        return true;
    }
    
    public function getName(): string {
        return "銀行振込 ({$this->bankName})";
    }
    
    public function isAvailable(): bool {
        return true;
    }
}

/**
 * DigitalWalletクラス
 */
class DigitalWallet implements PaymentMethod {
    private string $provider;
    private string $accountId;
    private int $balance;
    
    public function __construct(string $provider, string $accountId, int $balance) {
        $this->provider = $provider;
        $this->accountId = $accountId;
        $this->balance = $balance;
    }
    
    public function pay(int $amount): bool {
        if ($amount > $this->balance) {
            echo "ウォレット残高不足\n";
            return false;
        }
        
        echo "電子ウォレット({$this->provider})で" . number_format($amount) . "円支払いました\n";
        $this->balance -= $amount;
        return true;
    }
    
    public function getName(): string {
        return "電子ウォレット ({$this->provider})";
    }
    
    public function isAvailable(): bool {
        return $this->balance > 0;
    }
    
    public function addBalance(int $amount): void {
        $this->balance += $amount;
    }
}

/**
 * PaymentProcessorクラス
 */
class PaymentProcessor {
    private array $paymentMethods = [];
    private array $paymentHistory = [];
    
    public function __construct() {
        $this->paymentMethods = [];
        $this->paymentHistory = [];
    }
    
    public function addPaymentMethod(PaymentMethod $method): void {
        $this->paymentMethods[] = $method;
    }
    
    public function processPayment(PaymentMethod $method, int $amount): bool {
        if (!$method->isAvailable()) {
            echo "支払い方法は利用できません\n";
            return false;
        }
        
        $result = $method->pay($amount);
        
        $this->paymentHistory[] = [
            'method' => $method->getName(),
            'amount' => $amount,
            'success' => $result,
            'timestamp' => date('Y-m-d H:i:s')
        ];
        
        return $result;
    }
    
    public function listAvailableMethods(): array {
        return array_filter($this->paymentMethods, fn($m) => $m->isAvailable());
    }
    
    public function showPaymentHistory(): void {
        foreach ($this->paymentHistory as $i => $record) {
            $status = $record['success'] ? '成功' : '失敗';
            echo ($i + 1) . ". {$record['timestamp']} - {$record['method']}" . 
                 number_format($record['amount']) . "円 ({$status})\n";
        }
    }
}

// 動作確認
$processor = new PaymentProcessor();

$creditCard = new CreditCard("1234567890123456", "12/28", "123");
$bankTransfer = new BankTransfer("三菱UFJ銀行", "123-456-789");
$wallet = new DigitalWallet("PayPal", "user@example.com", 50000);

$processor->addPaymentMethod($creditCard);
$processor->addPaymentMethod($bankTransfer);
$processor->addPaymentMethod($wallet);

echo "=== 利用可能な支払い方法 ===\n";
foreach ($processor->listAvailableMethods() as $method) {
    echo "- " . $method->getName() . "\n";
}

echo "\n=== 支払いテスト ===\n";
$processor->processPayment($creditCard, 5000);
$processor->processPayment($wallet, 3000);

echo "\n=== 支払い履歴 ===\n";
$processor->showPaymentHistory();

?>

### 解説

#### 1. インターフェースの利点

**ポリモーフィズムの実現**:
```php
interface PaymentMethod {
    public function pay(int $amount): bool;
}

// 同じインターフェースを実装するクラスは同じように扱える
function process(PaymentMethod $method, int $amount) {
    $method->pay($amount);  // クラスに応じて異なる振る舞い
}
```

**利点**:
- 異なる実装を統一的に扱える
- 新しい支払い方法を追加しやすい
- テストが容易（モックを使える）

#### 2. 実装のポイント

**各クラスの特徴**:
- `CreditCard`: 与信枠チェック
- `BankTransfer`: 口座残高チェック
- `DigitalWallet`: 残高管理

**共通のインターフェース**:
- `pay()`: 支払い実行
- `getName()`: 表示名
- `isAvailable()`: 利用可能チェック

---
## 課題3: 割引計算システム（上級レベル）- 解説

### 戦略パターン（Strategy Pattern）

In [ ]:
<?php

// 割引戦略の実装例

class PercentageDiscount extends AbstractDiscount {
    private int $percentage;
    private int $minAmount;
    
    public function __construct(string $name, int $percentage, int $minAmount, int $priority) {
        parent::__construct($name, $priority);
        $this->percentage = $percentage;
        $this->minAmount = $minAmount;
    }
    
    public function appliesTo(ShoppingCart $cart): bool {
        return $cart->getSubtotal() >= $this->minAmount;
    }
    
    public function calculate(ShoppingCart $cart): int {
        return (int)floor($cart->getSubtotal() * $this->percentage / 100);
    }
}

class CategoryDiscount extends AbstractDiscount {
    private string $category;
    private int $percentage;
    
    public function __construct(string $name, int $percentage, int $priority, string $category) {
        parent::__construct($name, $priority);
        $this->percentage = $percentage;
        $this->category = $category;
    }
    
    public function appliesTo(ShoppingCart $cart): bool {
        return !empty($cart->getItemByCategory($this->category));
    }
    
    public function calculate(ShoppingCart $cart): int {
        $total = 0;
        foreach ($cart->getItemByCategory($this->category) as $item) {
            $total += (int)floor($item->getSubtotal() * $this->percentage / 100);
        }
        return $total;
    }
}

// DiscountCalculatorの実装
class DiscountCalculator {
    private array $strategies = [];
    
    public function addStrategy(DiscountStrategy $strategy): void {
        $this->strategies[] = $strategy;
    }
    
    public function calculateApplicableDiscounts(ShoppingCart $cart): array {
        $applicable = [];
        
        foreach ($this->strategies as $strategy) {
            if ($strategy->appliesTo($cart)) {
                $amount = $strategy->calculate($cart);
                $applicable[] = [
                    'name' => $strategy->getName(),
                    'amount' => $amount,
                    'priority' => $strategy->getPriority()
                ];
            }
        }
        
        // 優先度順にソート
        usort($applicable, fn($a, $b) => $b['priority'] <=> $a['priority']);
        
        return $applicable;
    }
    
    public function calculateBestDiscount(ShoppingCart $cart): int {
        $discounts = $this->calculateApplicableDiscounts($cart);
        
        if (empty($discounts)) {
            return 0;
        }
        
        // 最も割引額が大きいもの
        return max(array_column($discounts, 'amount'));
    }
    
    public function calculateWithStacking(ShoppingCart $cart, bool $allowMemberStacking = true): int {
        $discounts = $this->calculateApplicableDiscounts($cart);
        
        if (empty($discounts)) {
            return 0;
        }
        
        // 優先度順に処理
        $totalDiscount = 0;
        $memberApplied = false;
        
        foreach ($discounts as $discount) {
            // 会員割引は特別扱い
            if (str_contains($discount['name'], '会員')) {
                if ($allowMemberStacking) {
                    $totalDiscount += $discount['amount'];
                    $memberApplied = true;
                }
                continue;
            }
            
            // 会員割引以外は最初の1つのみ適用
            if (!$memberApplied) {
                $totalDiscount += $discount['amount'];
                break;
            }
        }
        
        return $totalDiscount;
    }
}

?>

### 解説

#### 1. 戦略パターン（Strategy Pattern）

**定義**:
- アルゴリズムの族を定義し、それぞれをカプセル化
- 実行時にアルゴリズムを交換可能

**構造**:
```php
DiscountStrategy (interface)
    ├── AbstractDiscount (abstract class)
    │   ├── PercentageDiscount
    │   ├── CategoryDiscount
    │   └── ...
    └── DiscountCalculator (context)
```

#### 2. 割引計算のロジック

**適用可能性のチェック**:
```php
public function appliesTo(ShoppingCart $cart): bool {
    return $cart->getSubtotal() >= $this->minAmount;
}
```

**割引額の計算**:
```php
public function calculate(ShoppingCart $cart): int {
    return (int)floor($cart->getSubtotal() * $this->percentage / 100);
}
```

#### 3. ポリモーフィズムの威力

**統一的な扱い**:
```php
// 全ての割引戦略をループ処理
foreach ($this->strategies as $strategy) {
    if ($strategy->appliesTo($cart)) {
        $discounts[] = $strategy->calculate($cart);
    }
}
```

**新しい割引の追加**:
```php
// 既存コードを変更せずに新しい戦略を追加
$calculator->addStrategy(new NewDiscount(...));
```

---
## ベストプラクティスまとめ

### 1. 継承よりコンポジション

**継承の問題点**:
- 深い階層は理解しにくい
- 多重継承ができない
- 親クラスの変更が子クラスに影響

**コンポジションの推奨**:
```php
// 悪い例：深い継承
class Administrator extends Moderator extends User extends Entity {}

// 良い例：コンポジション
class User {
    private Role $role;
}
```

### 2. リスコフの置換原則

**原則**:
- 子クラスは親クラスの代わりになれるべき
- 親クラスの契約を破ってはいけない

```php
// 良い例
class Square extends Rectangle {
    public function setSide(int $side): void {
        $this->setWidth($side);
        $this->setHeight($side);
    }
}

// 悪い例：Rectangleの挙動を変えてしまう
class Square extends Rectangle {
    public function setWidth(int $width): void {
        $this->width = $width;
        $this->height = $width;  // Rectangleの挙動を破壊
    }
}
```

### 3. インターフェース分離の原則

**大きなインターフェースを分割**:
```php
// 悪い例：1つの大きなインターフェース
interface User {
    public function save(): bool;
    public function delete(): bool;
    public function sendEmail(): bool;
    public function validate(): bool;
}

// 良い例：単一責任のインターフェース
interface Repository {
    public function save(): bool;
    public function delete(): bool;
}

interface Notifier {
    public function sendEmail(): bool;
}

interface Validatable {
    public function validate(): bool;
}
```

### 4. 依存性逆転の原則

**具象クラスではなく抽象に依存**:
```php
// 悪い例：具象クラスに依存
class OrderProcessor {
    private CreditCardPayment $payment;
}

// 良い例：インターフェースに依存
class OrderProcessor {
    public function __construct(
        private PaymentMethod $payment  // どの支払い方法でもOK
    ) {}
}
```

### 5. Web開発での応用

**Laravelの設計**:
```php
// インターフェース
interface UserRepositoryInterface {
    public function find(int $id): ?User;
    public function save(User $user): bool;
}

// 実装
class EloquentUserRepository implements UserRepositoryInterface {
    public function find(int $id): ?User {
        return User::find($id);
    }
    
    public function save(User $user): bool {
        return $user->save();
    }
}

// サービスクラスはインターフェースに依存
class UserService {
    public function __construct(
        private UserRepositoryInterface $repository
    ) {}
}
```

### 🎯 次のステップ

第11-14章を完了しました！オブジェクト指向プログラミングの基礎をマスターしました。

**次の学習ステップ**:
1. 名前空間とオートローディング
2. 例外処理
3. データベース操作（PDO）
4. Laravelフレームワーク
5. テスト（PHPUnit）

## 🎉 おめでとうございます！

PHP学習コースの中級レベルを完了しました！

**習得したスキル**:
- 連想配列の高度な操作
- クラスとオブジェクト
- カプセル化とアクセス修飾子
- 継承とポリモーフィズム
- インターフェースと抽象クラス
- デザインパターンの基礎